# Submitted by:
 
* Name 1: 
* Name 2:

## To be submitted by: Unknown yet

# paint_the_circle Project

This project identifies and paints circles given in RGB images. Specifically, each input image contains a rectangle, a triangle and a circle. The image is fed into a neural netork, whose outputs are the center and radius of the circle within the image. The domain considered is "[0,1]x[0,1] "box". PyTorch is the platform used. The goal is to paint the circle.

### **You are given the following files:**
  Three directories named "train", "validation", and "test" each containing the following:
  1. a directory called "images" with RGB png files of size 128x128 (so image that is read has shape (3,128,128))
  2. a file "labels.txt" containing the center points (as x,y) and radius of the circles for all images in "images" directory

Since this problem is somewhat open ended, you are given a suggested structure and some functionality that is already implemented for you (see below). You are not at all obigated to follow this structure or use the code given. 

**Note:** 
- "network" and "model" are used interchangeably throughout this documentation.
 
 
### **General structure (in order):**
 1. DataSet and DataLoader            - this is how you load data into the network **(given to you)**
 2. Neural Network definition         - the class that defines your model   **(need to implement)**
 3. calculate model parameters number - useful to get a feel for network's size **(given to you)**
 4. loss function definition          - to be used in training **(need to implement)**
 5. create an optimizer               - choose your optimizer **(need to implement)**
 6. estimate number of ops per forward feed - the bigger this is the slower the training **(given to you)**
 7. estimate ops per forward feed     - use function given to you to estimate this **(implement for convenience)**
 8. view images and target and net labels - example code to help you see how to use loaders **(given to you)**
 9. validate_model                   - returns avg loss per image for a model and loader  **(need to implement)**
 10. train_model                      - trains the network **(need to implement)**
 11. plot train/validaion losses      - visualizing train and validation losses from training **(given to you)**
 12. save/load model                  - Allows you to save and load model to disk for later use **(given to you)**
 13. visualizing images    - Painting circles prodiced by network on images from a given loader **(given to you)**
 
 
### ** What you need to do **
You have 5 (and a half...) things to do:
 1. Create a CircleNet class, which is your network model. This is a key component. The output of your model should be 3 numbers that represent the the center and radius of the circle in the input image fed into your network. You should keep in mind the number of parameters in your model. If there are too many, it may overfit (and take longer to run). If there are too few, you it may not be able to learn the task needed. A typical structure would have convolutional layers first and fully connected at the end, thus reducing number of parameteres. Consider which activation function you want to use, and whether or not you wish to use batch normalization or dropout. Pooling layers are also possible. Be creative.
 
 2. Create a loss function. This is another key component as it defines what it means for two circles (the true circle and its estimation) to be similar or not. Namely, two circles (represented by two centers and two radii: ground truth ("labels") and network outputs ("outputs")), whose images look similar should have a smaller loss than two circles whose images look less simialr. Think about how you would quantify "closeness"/"similarity" of circles.

 3. Choose an optimizer. Look here for some ideas: https://pytorch.org/docs/master/optim.html
 
 4. (This is the "half" thing to do). For your conveinece you may want to use the function calc_ops() that is given to you to calculate an estimate of the number of operations that your network does per feed forward. To do this, you need to enter your own network structure. An example of an arbitrary network is privided to you.
 
 5. Create a validate_model function that assesses (tests) the performance of a model. It returns the average loss per image for a given loader. It can be run on any set of data (train, validation, or test). You may want to run this function on validation set (loader) from within the train function (see below) train after each epoch so as to see how loss behaves on validation during the training process.
 
 6. Create a train_model function that trains model. This function updates the following parameteres: model, train_losses, and validation_losses. Model is updated simply by the training processes when optimzer.step() is called. The other two parameteres are lists that hold the average loss per image for the corresponding data (train or validation). After every epoch (i.e., iteration that goes over the entire train data), you should save to these lists the average loss per image for the corresponding data loader. These lists are useful in that you can plot them (functionality given to you) and observe how model behaves. Observe: this fucnton returs nothing, however, it updates parameteres by reference. Specifically, model is updated (trained), and so are train and validation losses values.
 
**Note:** You are given three sets of data: trian, validation, and test. It is recommended that test not be touched until the very end, and validation be used to get a sense of your network's performance. 

### ** Running on a GPU: **
It is not necessary to use a GPU for this project. However, if you choose to do so, you will gain a major speedup to your training, which will save you much time. The code given to you identifies the hardware used and will  automatically run on either a GPU or CPU. 

### ** Useful links: **
1. PyTorch master tutorial - VERY useful: https://pytorch.org/docs/master/nn.html
2. PyTorch optimizers: https://pytorch.org/docs/master/optim.html
3. A list of possible reasons why things go wrong: https://blog.slavv.com/37-reasons-why-your-neural-network-is-not-working-4020854bd607#74de

### ** Final tips: **
Use the Internet! Things will not work first time. You will get strange error messages. Google them up. The web is  great resource for tackling problems ranging from python error messages, to things not doing what you'd like them to do.


### ** Submission Instructions**
The project is to be submittd in teams as in the homework. You need to submit the following three files:
1. model.dat                            - This is your saved model 
2. paint_that_circle.ipynb              - This is this notebook containing all of your work
3. model.py                             - A file containing your CircleNet class only. 

Before you submit, run "check_before_submission.ipynb" to make sure your model could be properly tested. See instructions for running this notebook inside.

Make sure your names appear at the top of this notebook in the appropriate place.

### **GOOD LUCK!**

In [4]:
import numpy as np
import PIL.Image as Image
from PIL import ImageDraw
import torch.nn.functional as F
from torch.optim import Adam

import torch.nn as nn

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import os
import glob
import datetime
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Dataset class definition (given to you) 

In [5]:
class ShapesDataset(Dataset):
    
    def __init__(self, dataset_dir):
        """
        Initializing dataset by generating a dicitonary of labels, where an image file name is the key 
        and its labels are the contents of that entry in the dictionary. Images are not loaded. This way it
        is possible to iterate over arbitrarily large datasets (limited by labels dicitonary fitting 
        in memory, which is not a problem in practice)
        
        Args:
            dataset_dir : path to directory with images and labels. In this directory we expect to find
                          a directory called "images" containing the input images, and a file called 
                          "labels.txt" containing desired labels (coefficients)
        """
        
        self.dataset_dir = dataset_dir
        self.labels_dict = self.gen_labels_dict()
        self.images_keys = list(self.labels_dict)  # getting the keys of the dictionary as list
        self.images_keys.sort()                    # sorting so as to have in alphabetical order 

    def __len__(self):
        return len(self.labels_dict)

    def __getitem__(self, index):    
        """
        This funtion makes it possible to iterate over the ShapesDataset
        Args:
            index: running index of images
            
        Returns:
            sample: a dicitionary with three entries:
                    1. 'image'  contains the image
                    2. 'labels' contains labels (coeffs) corresponding to image
                    3. 'fname'  contains name of file (image_key) - may be useful for debugging
        """
        image_key = self.images_keys[index]     # recall - key is the file name of the corresponding image
        image = np.array(Image.open(image_key)) # image has shape: (128, 128, 3)
        image = image/255.0                     # simple normalization - just to maintain small numbers
        image = np.transpose(image, (2, 0, 1))  # network needs RGB channels to be first index
        labels = self.labels_dict[image_key]
        sample = {'image': image, 'labels': labels, 'fname':image_key}
        
        return sample
    
    
    def gen_labels_dict(self):
        """
        This fucntion generates a dictionary of labels
        
        Returns:
            labels_dict: the key is image file name and the value is the corresponding 
            array of labels  
        """
        
        labels_fname = self.dataset_dir + "/labels.txt"
        labels_dict = {}
        with open(labels_fname, "r") as inp:
            for line in inp:
                line = line.split('\n')[0]                                      # remove '\n' from end of line 
                line = line.split(',')
                key  = self.dataset_dir + '/images/' + line[0].strip() + ".png" # image file name is the key
                del line[0]
                
                list_from_line = [float(item) for item in line]
                labels_dict[key] = np.asarray(list_from_line, dtype=np.float32)
                        
        return labels_dict             


# Create Data Loaders (given to you) 

In [6]:
train_dir      = "./train/"
validation_dir = "./validation/"
test_dir       = "./test/"


train_dataset = ShapesDataset(train_dir)

train_loader = DataLoader(train_dataset, 
                          batch_size=32,
                          shuffle=True)



validation_dataset = ShapesDataset(validation_dir)

validation_loader = DataLoader(validation_dataset, 
                               batch_size=1,
                               shuffle=False)



test_dataset = ShapesDataset(test_dir)

test_loader = DataLoader(test_dataset, 
                          batch_size=1,
                          shuffle=False)


print("train loader examples     :", len(train_dataset)) 
print("validation loader examples:", len(validation_dataset))
print("test loader examples      :", len(test_dataset))

train loader examples     : 1000
validation loader examples: 100
test loader examples      : 100


# Neural Network class definition

In [28]:
class Unit(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(Unit,self).__init__()
        

        self.conv = nn.Conv2d(in_channels=in_channels,kernel_size=3,out_channels=out_channels,stride=1,padding=1)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU()

    def forward(self,input):
        output = self.conv(input)
        output = self.bn(output)
        output = self.relu(output)

        return output
    
class CircleNet(nn.Module):    # nn.Module is parent class  
    def __init__(self):
        super(CircleNet, self).__init__()  #calls init of parent class
                

        #----------------------------------------------
        # implementation needed here 
        #----------------------------------------------
        #Create 14 layers of the unit with max pooling in between
        self.unit1 = Unit(in_channels=3,out_channels=32)
        self.unit2 = Unit(in_channels=32, out_channels=32)
        self.unit3 = Unit(in_channels=32, out_channels=32)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.unit4 = Unit(in_channels=32, out_channels=64)
        self.unit5 = Unit(in_channels=64, out_channels=64)
        self.unit6 = Unit(in_channels=64, out_channels=64)
        self.unit7 = Unit(in_channels=64, out_channels=64)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.unit8 = Unit(in_channels=64, out_channels=128)
        self.unit9 = Unit(in_channels=128, out_channels=128)
        self.unit10 = Unit(in_channels=128, out_channels=128)
        self.unit11 = Unit(in_channels=128, out_channels=128)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.unit12 = Unit(in_channels=128, out_channels=128)
        self.unit13 = Unit(in_channels=128, out_channels=128)
        self.unit14 = Unit(in_channels=128, out_channels=128)

        self.avgpool = nn.AvgPool2d(kernel_size=4)
        
        #Add all the units into the Sequential layer in exact order
        self.net = nn.Sequential(self.unit1, self.unit2, self.unit3, self.pool1, self.unit4, self.unit5, self.unit6
                                 ,self.unit7, self.pool2, self.unit8, self.unit9, self.unit10, self.unit11, self.pool3,
                                 self.unit12, self.unit13, self.unit14, self.avgpool)

        self.fc = nn.Linear(in_features=128,out_features=num_classes)


    
    
                
    def forward(self, x):
        """
        Feed forward through network
        Args:
            x - input to the network
            
        Returns "out", which is the network's output
        """
        
        #----------------------------------------------
        # implementation needed here 
        #----------------------------------------------
        output = self.net(input)
        output = output.view(-1,128)
        output = self.fc(output)
        return output                
             
             
   

# Loss function definition

In [29]:
def my_loss(outputs, labels):
    
    """
    Args:
        outputs - output of network ([batch size, 3]) 
        labels  - desired labels  ([batch size, 3])
    """
    
    loss = torch.zeros(1, dtype=torch.float, requires_grad=True)
    loss = loss.to(device)
    
    #----------------------------------------------
    # implementation needed here 
    #----------------------------------------------
    loss = torch.mean((outputs - labels)**2)
    # Observe: If you need to iterate and add certain values to loss defined above
    # you cannot write: loss +=... because this will raise the error: 
    # "Leaf variable was used in an inplace operation"
    # Instead, to avoid this error write: loss = loss + ...  
    
                                      
    return loss


# Get number of trainable parameters (given to you)  

In [30]:
def get_train_params_num(model):
    """
    This fucntion returns the number of trainable parameters of neural network model
    You may want to call it after you create your model to see how many parameteres the model has
    Args:
        model - neural net to examine
    """
    
    #filter given iterable with a function that tests each element in the iterable to be true or not
    model_parameters = filter(lambda p: p.requires_grad == True, model.parameters()) 
    params_num = sum([np.prod(p.size()) for p in model_parameters])
    return params_num


# Model creation and choice of optimizer

In [31]:
cuda_avail = torch.cuda.is_available()
model = CircleNet.to(device)
print ("Number of model trainable parameters:", get_train_params_num(model))

if cuda_avail:
    model.cuda()

#Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()
#----------------------------------------------
#  Choose your optimizer:
#  implementation needed here 
#----------------------------------------------
# Create a learning rate adjustment function that divides the learning rate by 10 every 30 epochs
def adjust_learning_rate(epoch):
    lr = 0.001

    if epoch > 180:
        lr = lr / 1000000
    elif epoch > 150:
        lr = lr / 100000
    elif epoch > 120:
        lr = lr / 10000
    elif epoch > 90:
        lr = lr / 1000
    elif epoch > 60:
        lr = lr / 100
    elif epoch > 30:
        lr = lr / 10

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


AttributeError: 'torch.device' object has no attribute '_apply'

# Get an estimate number of operations (given to you)

In [17]:
def calc_ops(inp_size, net_struct):
    """
    Calculates a rough number of operations for a given network topology
    Args:
        inp_size - (W,H) of input 
        net_struct - list of tuples describing structure of network. 
        
        Example:
         (('conv2d', (3, 8, 3, 1, 0)),  # cin, cout, kernel, stride, pad
          ('conv2d': 83, 8, 3, 1, 0)),
          ('MaxPool2d', (2,2)),         # kernel, stride
          ('fc': (64, 8)),      
          ('fc': (8, 4)))
         
    """
    
    ops = 0
    W, H = inp_size
    for curr_item in net_struct:
        if curr_item[0] == 'conv2d':
            cin = curr_item[1][0]
            cout = curr_item[1][1]
            kernel = curr_item[1][2]
            stride = curr_item[1][3]
            pad = curr_item[1][4]
            W = (W +2*pad - kernel)/stride + 1
            H = (H +2*pad - kernel)/stride + 1
            curr_ops = (W*H*cin*cout*kernel*kernel)/stride
            ops += curr_ops
            print (curr_item, ":",  "{:,}".format(int(curr_ops)))
        elif curr_item[0] == 'MaxPool2d':
            kernel = curr_item[1][0]
            stride = curr_item[1][1]
            W = (W - kernel)/stride + 1
            H = (H - kernel)/stride + 1
        else:
            curr_ops = curr_item[1][0] * curr_item[1][1]
            ops += curr_ops
            print (curr_item, ":",  "{:,}".format(int(curr_ops)))
            
    return int(ops)

# Check rough number of ops for network (for your convenience)

In [18]:
inp_size = (128,128)

# place your network ropology in example_net below to obtain an estimated number of operations for your network
example_net = (('conv2d', (3, 3, 3, 1, 1)),
               ('MaxPool2d', (2,2)),
               ('conv2d', (3, 3, 3, 1, 1)),
               ('MaxPool2d', (2,2)),
               ('fc', (2883, 4)))


              
ops = calc_ops(inp_size, example_net)
print()
print("Total ops: {:,}".format(ops))

('conv2d', (3, 3, 3, 1, 1)) : 1,327,104
('conv2d', (3, 3, 3, 1, 1)) : 331,776
('fc', (2883, 4)) : 11,532

Total ops: 1,670,412


# View images, target circle labels and  network outputs (given to you)

In [19]:
"""
View first image of a given number of batches assuming that model has been created. 
Currently, lines assuming model has been creatd, are commented out. Without a model, 
you can view target labels and the corresponding images.
This is given to you so that you may see how loaders and model can be used. 
"""

loader = train_loader # choose from which loader to show images
bacthes_to_show = 2
with torch.no_grad():
    for i, data in enumerate(loader, 0): #0 means that counting starts at zero
        inputs = (data['image']).to(device)   # has shape (batch_size, 3, 128, 128)
        labels = (data['labels']).to(device)  # has shape (batch_size, 3)
        img_fnames = data['fname']            # list of length batch_size
        
        #outputs = model(inputs.float())
        img = Image.open(img_fnames[0])
        
        print ("showing image: ", img_fnames[0])
        
        labels_str = [ float(("{0:.2f}".format(x))) for x in labels[0]]#labels_np_arr]
        
        #outputs_np_arr = outputs[0] # using ".numpy()" to convert tensor to numpy array
        #outputs_str = [ float(("{0:.2f}".format(x))) for x in outputs_np_arr]
        print("Target labels :", labels_str )
        #print("network coeffs:", outputs_str)
        print()
        #img.show()
        
        if (i+1) == bacthes_to_show:
            break
        

showing image:  ./train//images/0607.png
Target labels : [0.7, 0.51, 0.18]

showing image:  ./train//images/0969.png
Target labels : [0.53, 0.17, 0.16]



# Validate model function

In [22]:
def validate_model(model, loader):
    """
    This function parses a given loader and returns the avergae (per image) loss 
    (as defined by "my_loss") of the entire dataset associated with the given loader.
    
    Args:
        model  - neural network to examine
        loader - where input data comes from (train, validation, or test)
        
    returns:
        average loss per image in variable named "avg_loss"
    """

    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
                  # (dropout is set to zero)

    #----------------------------------------------
    # implementation needed here 
    #----------------------------------------------

    for i, (images, labels) in enumerate(loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        print('Validation-loss',t, loss.item(),end=' ')


    return avg_loss
    

# train model function

In [26]:
def train_model(model,
                optimizer,
                train_loader,
                validation_loader,
                train_losses,
                validation_losses,
                epochs=1):
    
    """
    Trains a neural network. 
    Args:
        model               - model to be trained
        optimizer           - optimizer used for training
        train_loader        - loader from which data for training comes 
        validation_loader   - loader from which data for validation comes (maybe at the end, you use test_loader)
        train_losses        - adding train loss value to this list for future analysis
        validation_losses   - adding validation loss value to this list for future analysis
        epochs              - number of runs over the entire data set 
    """
    
    #----------------------------------------------
    # implementation needed here 
    #----------------------------------------------
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_acc = 0.0
        train_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # Move images and labels to gpu if available
            if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

            # Clear all accumulated gradients
            optimizer.zero_grad()
            # Predict classes using images from the test set
            outputs = model(images)
            # Compute the loss based on the predictions and actual labels
            loss = loss_fn(outputs, labels)
            # Backpropagate the loss
            loss.backward()

            # Adjust parameters according to the computed gradients
            optimizer.step()

            train_loss += loss.cpu().data[0] * images.size(0)
            _, prediction = torch.max(outputs.data, 1)
            
            train_acc += torch.sum(prediction == labels.data)

        # Call the learning rate adjustment function
        adjust_learning_rate(epoch)

        # Compute the average acc and loss over all 50000 training images
        train_acc = train_acc / 50000
        train_loss = train_loss / 50000

        # Evaluate on the test set
        test_acc = test()

        # Save the model if the test acc is greater than our current best
        if test_acc > best_acc:
            save_models(epoch)
            best_acc = test_acc

        # Print the metrics
        print("Epoch {}, Train Accuracy: {} , TrainLoss: {} , Test Accuracy: {}".format(epoch, train_acc, train_loss,test_acc))

 
    return
    


# Actual train (given to you)

In [27]:
# Using two lists (train_losses, validation_losses) containing history of losses 
# (i.e., loss for each training epoch) for train and validation sets. 
# If thess are not defined, we define them. Otherwise, the function train_model
# updates these two lists (by adding loss values when it is called for further training) 
# in order to be able to visualize train and validation losses


if not 'train_losses' in vars():
    train_losses = []
if not 'validation_losses' in vars():
    validation_losses = []


train_model(model, 
            optimizer,
            train_loader, 
            validation_loader, 
            train_losses, 
            validation_losses,
            epochs=2)


NameError: name 'model' is not defined

# Plot losses from training process (given to you)

In [ ]:
import matplotlib.pyplot as plt


iteration = np.arange(0., len(train_losses))
plt.plot(iteration, train_losses, 'g-', iteration, validation_losses, 'r-')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()
    


# Save/Load Model (given to you)

In [ ]:
def save(model, train_losses, validation_losses, save_dir):
    """
    saving model, train losses, and validation losses
    Args:
        model              - NN to be saved
        train_losses       - history of losses for training dataset
        validation_losses  - history of losses for validation dataset
        save_dir           - directory where to save the above
    """
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    files = glob.glob(save_dir + '*')
    for f in files:
        os.remove(f) 
    
    
    torch.save(model, save_dir + "/model.dat")
    
    train_losses_f = open(save_dir + "/train_losses.txt", "wt")
    train_losses_f.writelines( "%.3f\n" % item for item in train_losses)
    
    validation_losses_f = open(save_dir + "/validation_losses.txt", "wt")
    validation_losses_f.writelines( "%.3f\n" % item for item in validation_losses)

    return
   

def load(save_dir):
    """
    loading model, train losses, and validation losses
    Args:
       save_dir  - dir name from where to load 
    """
    
    model = torch.load(save_dir + "/model.dat") 
    
    train_losses_f = open(save_dir + "/train_losses.txt", "rt")
    train_losses   = train_losses_f.readlines()
    train_losses   = [float(num) for num in train_losses]
    
    validation_losses_f = open(save_dir + "/validation_losses.txt", "rt")
    validation_losses   = validation_losses_f.readlines()
    validation_losses   = [float(num) for num in validation_losses]
    
    return (model, train_losses, validation_losses)
   

# Example saving and loading

In [ ]:
# Create a directory, for example "./saves_12/", where you place your saved models

save(model, train_losses, validation_losses, "./saves_12/")

model, train_losses, validation_losses = load("./saves_12/")

# Paint circles on loader images (given to you)

In [ ]:
def paint_loader_circles(model, loader, out_dir):
    """
    This fucntion receives a model, a loader and an output directory. 
    For each image in the loader it paints a circle that the model identifies. 
    The images are saved in the given out_dir diretory. 
    Args:
        model   - network for idneitfying circles
        loader  - input data to use 
        out_dir - ouptut directory name (e.g.: 'draws/'). If directory does not exist, it is created.
                  If it exists, its files are deleted.
    """

    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
                  # (dropout is set to zero)

    k = 0
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    files = glob.glob(out_dir + '*')
    for f in files:
        os.remove(f) 
  
        
    for data in loader:
        # get inputs
        inputs = (data['image']).to(device)
        labels = (data['labels']).to(device)  # not using 
        img_fnames = data['fname'] 
      
        
        # forward
        outputs = model(inputs.float())
        curr_batch_size = np.shape(outputs)[0]
        image_size = np.shape(inputs[0])  # image_size = [3, w, h]
        _, width, height = image_size
        assert (width == height)
        
        for i in range (curr_batch_size): 
            x0 = (outputs[i, 0].item()) * width
            y0 = (1-outputs[i, 1].item()) * height
            r  = outputs[i, 2].item() * width #assume width=height here. Otherwise, circle becomes ellipse
   
            fname = img_fnames[i]
            k+=1
            print (str(k) + ".   " + fname)

            img = Image.open(fname)
            draw = ImageDraw.Draw(img, 'RGBA')
    
            draw.ellipse((x0 - r, y0 - r, x0 + r ,y0 + r), fill=(160, 64, 0, 90), outline=None)
    
            img.save(out_dir + fname.split('/')[-1])
    
        
    model.train()  #back to default
    return

# Example of how to paint circles produced by model

In [ ]:
# Painting circles on images from validation loader and placing them in directory "./validation/draw/". 
# Notice that if the painted circle is seen only partly, it means that it is not inside the 
# [0,1]x[0,1] "box", which is the domain considered.  This means that your model has siginificant error


draw_loader_circles(model, validation_loader, './validation/draw/')